# Homework

The goal of this homework is to create a regression model for predicting housing prices (column 'median_house_value').  

In this homework we'll again use the California Housing Prices dataset - the same one we used in homework 2 and 3.

You can take it from [Kaggle](https://www.kaggle.com/datasets/camnugent/california-housing-prices) or download using wget link mentioned below:

```
wget https://raw.githubusercontent.com/alexeygrigorev/datasets/master/housing.csv

```



In [37]:
#@ IMPORTING LIBRARIES AND DEPENDENCIES:
import re

import numpy as np
import pandas as pd
import seaborn as sns
import xgboost as xgb
import matplotlib.pyplot as plt

from tqdm.auto import tqdm
from sklearn.ensemble import RandomForestRegressor
from sklearn.exceptions import NotFittedError
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.tree import export_text, DecisionTreeRegressor
from sklearn.utils.validation import check_is_fitted

%matplotlib inline

In [5]:
#@ DOWNLOADING THE DATASET: UNCOMMENT BELOW:
!wget -nv https://raw.githubusercontent.com/alexeygrigorev/datasets/master/housing.csv

2022-10-10 23:25:15 URL:https://raw.githubusercontent.com/alexeygrigorev/datasets/master/housing.csv [1423529/1423529] -> "housing.csv.1" [1]


In [7]:
#@ READING DATASET:
PATH = "housing.csv"
select_cols = ["longitude", "latitude", "housing_median_age", "total_rooms", "total_bedrooms", "population", "households", 
               "median_income", "median_house_value", "ocean_proximity"]
df = pd.read_csv(PATH, usecols=select_cols)
df.total_bedrooms = df.total_bedrooms.fillna(0)

In [14]:
features = ['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 
               'median_income', 'ocean_proximity']
target = 'median_house_value_log'

- Apply the log transform to `median_house_value`. 
- Do train/validation/test split with 60%/20%/20% distribution.
- Use the `train_test_split` function and set the `random_state parameter` to 1.

In [9]:
df[target] = np.log1p(df.median_house_value.values)

In [10]:
def train_val_test_split(df, val_split=0.2, test_split=0.2, seed=None):

  # train_split = 1 - val_split - test_split

  # create splits
  df_full_train, df_test = train_test_split(df, test_size=test_split, random_state=seed)
  df_train, df_val = train_test_split(df_full_train, test_size=val_split/(1-test_split), random_state=seed)

  # return
  return (df_train, df_val, df_full_train, df_test)

In [11]:
df_train, df_val, df_full_train, df_test = train_val_test_split(df, seed=1)

- We will use `DictVectorizer` to turn train and validation into matrices.

In [13]:
def one_hot_encoding(dv, df, features, target):

  feat_dict = df[features].to_dict(orient='records')
  
  try:
    check_is_fitted(dv, attributes='feature_names_')
  except NotFittedError as e:
    dv.fit(feat_dict)
  
  X = dv.transform(feat_dict)
  y = df[target].values

  return (X, y)

In [15]:
dv = DictVectorizer(sparse=False)

X_train, y_train = one_hot_encoding(dv, df_train, features, target)
X_val, y_val = one_hot_encoding(dv, df_val, features, target)

**Question 1**

Let's train a decision tree regressor to predict the `median_house_value` variable.

Train a model with `max_depth=1`.

In [16]:
#@ TRAINING THE REGRESSION MODEL:
dt = DecisionTreeRegressor(max_depth=1)
dt.fit(X_train, y_train)

DecisionTreeRegressor(max_depth=1)

In [20]:
#@ INSPECTION:
print(export_text(dt, feature_names=dv.get_feature_names()))

|--- ocean_proximity=INLAND <= 0.50
|   |--- value: [12.31]
|--- ocean_proximity=INLAND >  0.50
|   |--- value: [11.61]



- Which feature is used for splitting the data?

- Answer: `ocean_proximity=INLAND`

**Question 2**

Train a random forest model with these parameters:

- `n_estimators=10`  
- `random_state=1`  
- `n_jobs=-1` (optional-to make training faster)

In [21]:
#@ TRAINING RANDOM FOREST MODEL:
dt = RandomForestRegressor(n_estimators=10, random_state=1, n_jobs=-1)
dt.fit(X_train, y_train)

RandomForestRegressor(n_estimators=10, n_jobs=-1, random_state=1)

In [28]:
#@ CALCULATING MEAN SQUARED ERROR:
y_pred = dt.predict(X_val)
rmse = mean_squared_error(y_val, y_pred)
    
print('%.2f' % (rmse))

0.06


- What's the RMSE of this model on validation?

- Answer: `0.06`

**Question 3**

Now, let's experiment with the `n_estimators` parameter.

- Try different values of this parameter from 10 to 200 with step 10.
- Set `random_state` to 1.
- Evaluate the model on the validation dataset.

In [35]:
#@ TRAINING THE RANDOM FOREST MODEL:
n_estimators = np.arange(10, 201, step=10)

for n_estimator in n_estimators: 
    dt = RandomForestRegressor(n_estimators=n_estimator, random_state=1, n_jobs=-1)
    dt.fit(X_train, y_train)

    y_pred = dt.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred)
    
    print('%4s -> %.3f' % (n_estimator, rmse))


  10 -> 0.060
  20 -> 0.057
  30 -> 0.056
  40 -> 0.055
  50 -> 0.054
  60 -> 0.054
  70 -> 0.054
  80 -> 0.054
  90 -> 0.054
 100 -> 0.054
 110 -> 0.054
 120 -> 0.054
 130 -> 0.054
 140 -> 0.054
 150 -> 0.054
 160 -> 0.054
 170 -> 0.054
 180 -> 0.054
 190 -> 0.054
 200 -> 0.054


- After which value of `n_estimators` does RMSE stop improving?

- Answer: `n_estimators = 50`

**Question 4**

Let's select the best `max_depth`:

- Try different values of `max_depth`: [10, 15, 20, 25].
- For each of these values, try different values of n_estimators from 10 till 200 (with step 10).
- Fix the random seed: `random_state=1`.

In [40]:
parameters = {'max_depth':[10, 15, 20, 25], 'n_estimators':np.arange(10, 201, step=10)}
rf = RandomForestRegressor(random_state=1, n_jobs=-1)
clf = GridSearchCV(rf, parameters, scoring='neg_root_mean_squared_error')
clf.fit(X_train, y_train)

GridSearchCV(estimator=RandomForestRegressor(n_jobs=-1, random_state=1),
             param_grid={'max_depth': [10, 15, 20, 25],
                         'n_estimators': array([ 10,  20,  30,  40,  50,  60,  70,  80,  90, 100, 110, 120, 130,
       140, 150, 160, 170, 180, 190, 200])},
             scoring='neg_root_mean_squared_error')

In [41]:
clf.best_params_

{'max_depth': 25, 'n_estimators': 170}

In [54]:
#@ TRAINING THE MODEL WITH DEPTH:
n_estimators = np.arange(10, 201, step=10)
depths = [10, 15, 20, 25]

for depth in depths:
    for n_estimator in n_estimators: 
    
        dt = RandomForestRegressor(n_estimators=n_estimator, max_depth=depth, random_state=1, n_jobs=-1)
        dt.fit(X_train, y_train)

        y_pred = dt.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred)
        
        print('%2s \t %4s \t %.3f' % (depth, n_estimator, rmse))


10 	   10 	 0.065
10 	   20 	 0.064
10 	   30 	 0.063
10 	   40 	 0.063
10 	   50 	 0.063
10 	   60 	 0.063
10 	   70 	 0.062
10 	   80 	 0.062
10 	   90 	 0.062
10 	  100 	 0.062
10 	  110 	 0.062
10 	  120 	 0.062
10 	  130 	 0.062
10 	  140 	 0.062
10 	  150 	 0.062
10 	  160 	 0.062
10 	  170 	 0.062
10 	  180 	 0.062
10 	  190 	 0.062
10 	  200 	 0.062
15 	   10 	 0.060
15 	   20 	 0.057
15 	   30 	 0.056
15 	   40 	 0.056
15 	   50 	 0.055
15 	   60 	 0.055
15 	   70 	 0.055
15 	   80 	 0.055
15 	   90 	 0.055
15 	  100 	 0.055
15 	  110 	 0.054
15 	  120 	 0.054
15 	  130 	 0.054
15 	  140 	 0.054
15 	  150 	 0.054
15 	  160 	 0.054
15 	  170 	 0.054
15 	  180 	 0.055
15 	  190 	 0.055
15 	  200 	 0.054
20 	   10 	 0.061
20 	   20 	 0.057
20 	   30 	 0.056
20 	   40 	 0.055
20 	   50 	 0.055
20 	   60 	 0.055
20 	   70 	 0.054
20 	   80 	 0.054
20 	   90 	 0.054
20 	  100 	 0.054
20 	  110 	 0.054
20 	  120 	 0.054
20 	  130 	 0.054
20 	  140 	 0.054
20 	  150 	 0.054
20 	  160 

- What's the best `max_depth`:

- Answer:

**Question 5**

We can extract feature importance information from tree-based models.

At each step of the decision tree learning algorith, it finds the best split. When doint it, we can calculate "gain" - the reduction in impurity before and after the split. This gain is quite useful in understanding what are the imporatant features for tree-based models.

In Scikit-Learn, tree-based models contain this information in the `feature_importances_` field.

For this homework question, we'll find the most important feature:

Train the model with these parametes:
- `n_estimators=10`,
- `max_depth=20`,
- `random_state=1`,
- `n_jobs=-1` (optional)

Get the feature importance information from this model

In [43]:
#@ TRAINING THE RANDOM FOREST MODEL:
rf = RandomForestRegressor(n_estimators=10, max_depth=20, random_state=1, n_jobs=-1)
rf.fit(X_train, y_train)

RandomForestRegressor(max_depth=20, n_estimators=10, n_jobs=-1, random_state=1)

In [48]:
dict(zip(dv.get_feature_names_out(), rf.feature_importances_))

{'households': 0.01707949729751655,
 'housing_median_age': 0.03293146678518498,
 'latitude': 0.10154547618368868,
 'longitude': 0.09632464688219677,
 'median_income': 0.36366102999455063,
 'ocean_proximity=<1H OCEAN': 0.0028781968739456333,
 'ocean_proximity=INLAND': 0.3109008414649182,
 'ocean_proximity=ISLAND': 0.000356806262945808,
 'ocean_proximity=NEAR BAY': 0.000421521148856216,
 'ocean_proximity=NEAR OCEAN': 0.0042237262548374226,
 'population': 0.029629759818677763,
 'total_bedrooms': 0.018239616242627415,
 'total_rooms': 0.021807414790053964}

- What's the most important feature?

- Answer: `median_income`

**Question 6**

Now let's train an XGBoost model! For this question, we'll tune the eta parameter:

- Install XGBoost.
- Create DMatrix for train and validation
- Create a watchlist
- Train a model with these parameters for 100 rounds:

```
xgb_params = {  
    'eta': 0.3,  
    'max_depth': 6,  
    'min_child_weight': 1,  

    'objective': 'reg:squarederror',
    'nthread': 8,

    'seed': 1,
    'verbosity': 1,
}
```



In [49]:
#@ CREATING THE DMARTIX:
features = dv.feature_names_

regex = re.compile(r"<", re.IGNORECASE)
features = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in features]

dtrain = xgb.DMatrix(X_train, label=y_train, feature_names=features)
dval = xgb.DMatrix(X_val, label=y_val, feature_names=features)

In [53]:
etas = [0.3, 0.1, 0.01]

for eta in etas:

    xgb_params = {
        'eta': eta, 
        'max_depth': 6,
        'min_child_weight': 1,
        
        'objective': 'reg:squarederror',
        'nthread': 8,
        
        'seed': 1,
        'verbosity': 1,
    }

    model = xgb.train(xgb_params, dtrain, num_boost_round=100)

    y_pred = model.predict(dval)
    rmse = mean_squared_error(y_val, y_pred)

    print('%2f \t %.3f' % (eta, rmse))

0.300000 	 0.051
0.100000 	 0.053
0.010000 	 18.190


- Now, change eta first to 0.1 and then to 0.01.

- Which eta leads to the best RMSE score on the validation dataset?

- Answer: `0.3`